# Which translation model is used when using an HuggingFace translation pipeline ?

For default, but from the transformer translation documentation in hugging face, I found that the Text-To-Text Transfer Transformer (T5) small from Google is used, especially the my_awesome_opus_books_model model - a fine-tuned version of t5-small on the opus_books dataset.

# What is the BLEU score achieved on the challenge set by an LLM ? by the translation pipeline ? How long does it take to translation the test set with a LLM ? With a specific model ?

<i> The ARC question set is partitioned into a Challenge Set and an Easy Set, where the Challenge Set contains only questions answered incorrectly by both a retrieval-based algorithm and a word co-occurence algorithm ...<i> https://til.hashnode.dev/running-an-llm-locally

In [1]:
from datasets import load_dataset, Dataset
from functools import partial
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from ctransformers import AutoModelForCausalLM
import torch
import pandas as pd
import os
import re
import evaluate

In [2]:
# get challenge set
df = pd.read_json( os.path.join(*["/home", "caegi", "Documents", "M1", "ML", 
                        "ML2", "TP4", "D17-1263.Attachment", "Challenge_set-v2hA.json"]), lines=True)

# I am removing brackets to limit the prompting and translation models' tomfoolery (it makes the BLEU score better)
def remove_brackets(s):
    return re.sub("\[|\]", '', s)

df["reference"] = df["reference"].apply(remove_brackets)
df["source"] = df["source"].apply(remove_brackets)

df.head()

,category_minor,reference,category_major,question,source,systems,id
0,"S-V agreement, across distractors",Les appels répétés de sa mère auraient dû nous...,Morpho-Syntactic,Is subject-verb agrement correct? (Possible in...,The repeated calls from his mother should have...,[{'output': 'Les appels répétés de sa mère aur...,S1a
1,"S-V agreement, across distractors",Le bruit soudain dans les chambres supérieures...,Morpho-Syntactic,Is subject-verb agrement correct? (Possible in...,The sudden noise in the upper rooms should hav...,[{'output': 'Le bruit soudain dans les chambre...,S1b
2,"S-V agreement, across distractors",Leurs échecs répétés à signaler le problème au...,Morpho-Syntactic,Is subject-verb agrement correct? (Possible in...,Their repeated failures to report the problem ...,[{'output': 'Leur échec répété à signaler le p...,S1c
3,"S-V agreement, through control verbs",Elle a demandé à son frère de ne pas se montre...,Morpho-Syntactic,Does the flagged adjective agree correctly wit...,She asked her brother not to be arrogant.,[{'output': 'Elle a demandé à son frère de ne ...,S2a
4,"S-V agreement, through control verbs",Elle a promis à son frère de ne pas être arrog...,Morpho-Syntactic,Does the flagged adjective agree correctly wit...,She promised her brother not to be arrogant.,[{'output': 'Elle a promis à son frère de ne p...,S2b


## LLM (text generation)

In [3]:
# https://github.com/yashar1908/Text-translation-using-Mistral-7B/blob/main/LLM_for_Translation.ipynb
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
model_type = "mistral", gpu_layers = 50)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
def get_translation_llm(s):
    prompt = f'Translate this Text from French to English:{s} Translation:'
    return llm(prompt, max_new_tokens=100, temperature=0.9, top_k=55, top_p=0.93, repetition_penalty=1.2)

In [5]:
%time predictions_llm = df["reference"].apply(get_translation_llm).to_list()

CPU times: user 7min 24s, sys: 27 s, total: 7min 51s
Wall time: 1min 59s


In [ ]:
references = df["source"].to_list()

# see some results
for i in range(3):
    print("\n", df["reference"][i])
    print("Gold Translation is: ", references[i])
    print("Prompting Translation is: ", predictions_llm[i])

In [17]:
sacrebleu = evaluate.load("sacrebleu")
print("text generation bleu score:", sacrebleu.compute(predictions=predictions_llm, references=references)["score"])

text generation bleu score: 50.25696048117239


## Translation Pipeline

In [9]:
translation_pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")

def get_translation_fr_en_model(s):
    return translation_pipe(s, max_length=100)[0]['translation_text']

In [10]:
%time predictions_fr_en_model = df["reference"].apply(get_translation_fr_en_model).to_list()

CPU times: user 1min 18s, sys: 148 ms, total: 1min 18s
Wall time: 19.6 s


In [16]:
print("fr_en_model bleu score:", sacrebleu.compute(predictions=predictions_fr_en_model, references=references)["score"])

fr_en_model bleu score: 60.56494528173656


## Conclusion

The blue score of the specialized machine translation model from french to english is better than the text generation model one, although the score of the LLM text generation model would be higher if I had more VRAM in my GPU to run a better model.

The time difference to get the translations is significant. As you can see in the cell 5 and 10, the machine translation model is around 6 times faster than the text generation model. (1min 18s vs 7min 51s)